In [ ]:
#import the required libraries, time, csv, datetime and os are preinstalled in python
# Import the paho.mqtt.client library for MQTT communication
# This library allows you to connect to an MQTT broker and send/receive messages.
# To install it, un cooment the line below and run, you may need to resarts once installed:
# pip install paho-mqtt
import paho.mqtt.client as mqtt
import time
import csv
# Import the requests library to make HTTP requests
# This library is used to interact with web APIs, like fetching weather and pollution data.
# To install it uncomment the line below and run, you may need to resarts once installed:
# pip install requests
import requests
import requests
from datetime import datetime
import os

# Get the user's home directory
home_dir = os.path.expanduser("~")
#accessible to all technologies - error handle
# Define the path to the Desktop of where the csv file will save (macOS/Linux: ~/Desktop, Windows: C:\Users\<User>\Desktop) 
if os.name == 'nt':  # For Windows
    desktop_path = os.path.join(home_dir, 'Desktop', 'Example.csv')
else:  # For macOS/Linux
    desktop_path = os.path.join(home_dir, 'Desktop', 'Example.csv')

# Print the Desktop path for debugging
print("CSV will be saved to:", desktop_path)

# MQTT broker and topics to subscribe to
broker = "https://test.mosquitto.org" #Example broker using test.mosquittos
topics = [
    ('8742time', 0),  # Arduino time
    ('5817iaq', 0),  # Indoor Air Quality
    ('4381comptemp', 0),  # Compensated Temperature
    ('5907comphum', 0),  # Compensated Humidity
    ('6574compgas', 0),  # Compensated Gas concentration
    ('3094MS501voc', 0),  # Volatile Organic Compounds (VOC)
    ('7624MS501c02', 0)   # CO2 concentration
]

# Define column titles for the CSV file
column_titles = [
    'Overall ID', 'Experiment ID', 'Location ID', 'Measurement ID', 'Experiment Condition', 
    'timestamp', 'time', 'iaq', 'Compensated Temperature / C', 'Compensated Humidity / %',
    'Compensated Gas / ppm', 'VOC / ppb', 'CO2 / ppm', 'Temperature (Weather)', 'Humidity (Weather)', 
    'Wind Speed (Weather)', 'AQI (Pollution)', 'CO Concentration', 'NO2 Concentration'
]

# Dictionary to hold the data received from MQTT broker
data = {}

# Initialize measurement_id to start fresh on each run - this will increase by one for each data set
measurement_id = 1

# Callback function for handling received MQTT messages
def on_message(client, userdata, message):
    """
    This function is called whenever a new message is received from the MQTT broker.
    It decodes the message payload and stores it in the `data` dictionary.
    
    Args:
    - client: The MQTT client instance.
    - userdata: Custom data passed to the callback function.
    - message: The message object containing the topic and payload.
    """
    topic = message.topic
    payload = message.payload.decode('utf-8')  # Decode message payload
    
    print(f"Received message: {payload} on topic {topic}")
    
    try:
        # Try to store the message as a float if possible
        data[topic] = float(payload)
    except ValueError:
        # If not a number, store it as a string - error handling 
        data[topic] = payload

def get_weather_and_pollution(api_key, city):
    """
    Fetches the weather and pollution data for a specific city using the OpenWeather API.
    
    Args:
    - api_key: Your OpenWeather API key.
    - city: The city name to fetch the weather data for.
    
    Returns:
    - A dictionary containing weather and pollution data if successful, otherwise None.
    """
    weather_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
    weather_response = requests.get(weather_url)
    #print error message if the weather and air pollution data cannot be recieved - error handling
    if weather_response.status_code == 200:
        weather_data = weather_response.json()
        city_name = weather_data['name']
        temperature = weather_data['main']['temp']
        humidity = weather_data['main']['humidity']
        wind_speed = weather_data['wind']['speed']
        latitude = weather_data['coord']['lat']
        longitude = weather_data['coord']['lon']

        # Get air pollution data
        pollution_url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={latitude}&lon={longitude}&appid={api_key}'
        pollution_response = requests.get(pollution_url)

        if pollution_response.status_code == 200:
            pollution_data = pollution_response.json()
            aqi = pollution_data['list'][0]['main']['aqi']  # Air Quality Index (AQI)
            components = pollution_data['list'][0]['components']
            
            return {
                'weather_temperature': temperature,
                'weather_humidity': humidity,
                'weather_wind_speed': wind_speed,
                'aqi': aqi,
                'co_concentration': components.get('co', None),
                'no2_concentration': components.get('no2', None),
            }
        else:
            print("Error retrieving air pollution data")
            return None
    else:
        print("Error retrieving weather data")
        return None

# Create an MQTT client instance
client = mqtt.Client()
client.on_message = on_message  # Set the message callback function

# Connect to the MQTT broker
client.connect(broker)

# Subscribe to each topic with QoS level 0
for topic, qos in topics:
    print(f"Subscribing to {topic}")
    client.subscribe(topic, qos)

# Start the MQTT client loop to receive messages
client.loop_start()

# Main loop to collect and save data
try:
    while True:
        # Fetch weather and pollution data every 60 seconds
        api_key = 'yourapikeyxxxxxx'  # Replace with your OpenWeather API key - instructions on how to get one are in my ReadMe
        city = 'liverpool'  # City to fetch weather and pollution data for
        weather_pollution_data = get_weather_and_pollution(api_key, city)

        # If the weather and pollution data was successfully fetched
        if weather_pollution_data:
            # Get the current timestamp (iOS-like format)
            ios_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            arduino_time = data.get('8742time', '')  # Get the Arduino time from the 'time' topic

            # Add weather and pollution data to the data dictionary
            data.update(weather_pollution_data)

            # Add static data fields (Experiment ID, Location ID, etc.) to geneate a unique ID for each data set - useful for data strorage and SQL querying
            experiment_id = '1' #change this depending on experiment number
            location_id = '1' #change this depending on Arduino location
            overall_id = f"{experiment_id}_{location_id}_{measurement_id}"

            data['Experiment ID'] = experiment_id
            data['Experiment Condition'] = 'Window Open' #change this depending on experiment condition 
            data['Measurement ID'] = str(measurement_id)
            data['Location ID'] = location_id
            data['Overall ID'] = overall_id  # Combine to create Overall ID

            # Try to open the CSV file in append mode and write the data - error handled
            try:
                with open(desktop_path, mode='a', newline='') as file:
                    writer = csv.DictWriter(file, fieldnames=column_titles)

                    # If the file is empty (first write), write the header
                    if file.tell() == 0:
                        writer.writeheader()

                    # Prepare the data row to write into the CSV
                    row = {
                        'Overall ID': overall_id,  # Add Overall ID as the first item in the row
                        'Experiment ID': experiment_id,
                        'Location ID': location_id,
                        'Measurement ID': str(measurement_id),
                        'Experiment Condition': 'Window Open',  # Static value
                        'timestamp': ios_timestamp,  # iOS timestamp
                        'time': arduino_time,  # Arduino time
                    }

                    # Add dynamic data for each topic (from MQTT)
                    row.update({
                        column_titles[i + 7]: data.get(topic, '') for i, (topic, _) in enumerate(topics[1:])
                    })
                    row.update({
                        'Temperature (Weather)': data['weather_temperature'],
                        'Humidity (Weather)': data['weather_humidity'],
                        'Wind Speed (Weather)': data['weather_wind_speed'],
                        'AQI (Pollution)': data['aqi'],
                        'CO Concentration': data['co_concentration'],
                        'NO2 Concentration': data['no2_concentration'],
                    })

                    # Write the data row to the CSV file
                    writer.writerow(row)

                    print(f"Data saved to CSV at {desktop_path}!")

                # Increment the Measurement ID for the next row so each measurement for a particular timestamp has a unique key
                measurement_id += 1
            #csv error handling
            except PermissionError as e:
                print(f"PermissionError: Unable to write to file {desktop_path}. Please check your file system permissions.")
            except Exception as e:
                print(f"An error occurred while writing to the file: {e}")
        
        # Sleep for 1 second to control rate of data fetching
        time.sleep(1)  # Sleep before fetching new weather/pollution data

except KeyboardInterrupt: #allow use to end data collection by interupting the keyboard - error handling
    print("Exiting...") 
    client.loop_stop()  # Stop the MQTT loop when exiting the program
    client.disconnect()  # Disconnect from the MQTT broker
